In [4]:
from google.cloud import storage

path_to_private_key = 'is3107-418903-a6116d294003.json'

storage_client = storage.Client.from_service_account_json(json_credentials_path=path_to_private_key)

In [6]:
from google.api_core.exceptions import Forbidden, NotFound

def download_blob(bucket_name,source_blob_name, destination_file_name):
    try:
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(source_blob_name)
        blob.download_to_filename(destination_file_name)
    except Forbidden as e:
        print(f"Access denied. You might have a billing or permissions issue: {e}")
    except NotFound as e:
        print(f"The requested file was not found: {e}")

    

# download models
download_blob('is3107-model', 'ml_model_test/modelGB.joblib', 'modelGB.joblib')
download_blob('is3107-model', 'ml_model_test/modelDT.joblib', 'modelDT.joblib')

# download model dataset
download_blob('is3107-model', 'ml_model_test/dataset.csv', 'dataset.csv')

In [7]:
import joblib
import pandas as pd

modelGB = joblib.load('modelGB.joblib')
modelDT = joblib.load('modelDT.joblib')
data = pd.read_csv('dataset.csv')

ModuleNotFoundError: No module named 'sklearn.ensemble._gb_losses'

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

x = data.drop('price')
y = data['price']

grid_searchGB = GridSearchCV(modelGB, param_grid, cv=3, scoring='accuracy')
grid_searchDT = GridSearchCV(modelDT, param_grid, cv=3, scoring='accuracy')


# Fit on your data
grid_searchGB.fit(x, y)
grid_searchDT.fit(x, y)


# Best model
best_modelGB = grid_searchGB.best_estimator_
best_modelDT = grid_searchDT.best_estimator_

## Cross Evaluation

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming you have a separate test set (data_features_test, data_labels_test)
predictionsGB = best_modelGB.predict(data_features_test)

# Evaluate the model
print("Accuracy:", accuracy_score(data_labels_test, predictionsGB))
print("Classification Report:\n", classification_report(data_labels_test, predictionsGB))
print("Confusion Matrix:\n", confusion_matrix(data_labels_test, predictionsGB))


# Assuming you have a separate test set (data_features_test, data_labels_test)
predictionsDT = best_modelGB.predict(data_features_test)

# Evaluate the model
print("Accuracy:", accuracy_score(data_labels_test, predictionsDT))
print("Classification Report:\n", classification_report(data_labels_test, predictionsDT))
print("Confusion Matrix:\n", confusion_matrix(data_labels_test, predictionsDT))

In [ ]:
from sklearn.model_selection import cross_val_score

# Cross-validate the model
scores = cross_val_score(best_modelGB, data_features, data_labels, cv=5)  # Using the full data for demonstration
print("Cross-Validation Accuracy Scores:", scores)
print("Mean CV Accuracy:", scores.mean())

# Cross-validate the model
scores = cross_val_score(best_modelDT, data_features, data_labels, cv=5)  # Using the full data for demonstration
print("Cross-Validation Accuracy Scores:", scores)
print("Mean CV Accuracy:", scores.mean())

# Predict on user input

In [ ]:
user_input = pd.read_csv('user_input.csv')
# do pre-precossing

predictions = best_modelDT.predict(user_input)

# Optionally, you can output these predictions to a DataFrame or directly use them
results_df = pd.DataFrame({
    'Predictions': predictions
})
print(results_df.head())